In [62]:
! pip install progressbar2

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import re
from progressbar import progressbar
import time
import random

### 상세 검색 페이지에서 파라미터 확인

http://dart.fss.or.kr/dsab002/main.do#  페이지 접속


'주식분할' 키워드로 검색


'검사' 메뉴에서 Form Data 확인


당일 날짜 설정

In [64]:
# 금일 날짜 설정

dt = datetime.utcnow() + timedelta(hours = 9)    # utc 시간에서 9시간 더함
today = dt.strftime('%Y%m%d')                  # str 형태로 변환
today

'20190105'

In [65]:
url = 'http://dart.fss.or.kr/dsab002/search.ax'

In [66]:
data = {
    'currentPage': '1',            # 페이지 1로 충분함
    'maxResults': '100',           # 15에서 100으로 변경
    'maxLinks': '10',
    'sort': 'date',
    'series': 'desc',
    'textCrpCik': '',
    'textCrpNm': '',
    'textPresenterNm': '',
    'startDate': '20160101',
    'endDate': today,         # 오늘 날짜로 변경
    'finalReport': 'recent',
    'typesOfBusiness': 'all',
    'corporationType': 'all',
    'closingAccountsMonth': 'all',
    'reportName': '주식분할'
    }

# headers = {
#     'Origin': 'http://dart.fss.or.kr',
#     'Accept-Encoding': 'gzip, deflate',
#     'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36',
#     'Content-Type': 'application/x-www-form-urlencoded',
#     'Accept': 'text/html, */*; q=0.01',
#     'Referer': 'http://dart.fss.or.kr/dsab002/main.do',
#     'X-Requested-With': 'XMLHttpRequest',
#     'Connection': 'keep-alive',
# }


#### 원하는 페이지 요청

In [70]:
response = requests.post(url, data = data)

In [69]:
response.text

'\r\n\r\n\r\n\r\n<div class="table_list" >\r\n\t<p class="page_count">\r\n\t\t<label for="maxResultsCb">조회건수</label>\r\n\t\t<select id="maxResultsCb" name="maxResultsCb" onChange="maxResultSet()" style="width: 50px;">\r\n\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\t<option value="15">15</option>\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\t<option value="30">30</option>\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\t<option value="50">50</option>\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\t\t<option value="100" selected>100</option>\r\n\t\t\t\r\n\t\t\t\r\n\t\t\r\n\t\t</select>\r\n\t</p>\r\n\t<p class="sort2">\r\n\t    \r\n\t    \t\r\n\t\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\r\n\t\t\r\n\t\t\t\r\n\t\t\t\t<a href="#date1" onclick="setOrder(date); return false;"><img name="sortImg" id="date" src="/images/sort/date/on_desc.gif" alt="접수일자 내림차순" /></a>\r\n\t\t\t\t<a href="#crp1" onclick="setOrder(crp); return false;"><img name="sortImg"  id="crp" src="/images/sort/crp/sort

#### 페이지 응답 여부 확인해보기

In [71]:
dff = pd.read_html(response.text)[0]

In [72]:
dff

,번호,공시대상회사,보고서명,제출인,접수일자,비고
0,1,방림,주식분할결정,방림,2018.12.04,NaN
1,2,리켐,주식분할결정,리켐,2018.11.13,NaN
2,3,중앙에너비스,[기재정정]주식분할결정,중앙에너비스,2018.10.30,NaN
3,4,포스링크,[기재정정]주식분할결정,포스링크,2018.10.19,NaN
4,5,인콘,[기재정정]주식분할결정,인콘,2018.09.11,NaN
5,6,메가스터디교육,[기재정정]주식분할결정,메가스터디교육,2018.08.22,NaN
6,7,모트렉스,[기재정정]주식분할결정,모트렉스,2018.08.09,NaN
7,8,바이오빌,[기재정정]주식분할결정,바이오빌,2018.08.07,NaN
8,9,대창솔루션,[첨부정정]주식분할결정,대창솔루션,2018.08.01,NaN
9,10,에이스침대,주식분할결정,에이스침대,2018.08.01,NaN


In [73]:
dff.shape

(100, 6)

In [74]:
soup = bs(response.text, 'html.parser')

In [75]:
soup


<div class="table_list">
<p class="page_count">
<label for="maxResultsCb">조회건수</label>
<select id="maxResultsCb" name="maxResultsCb" onchange="maxResultSet()" style="width: 50px;">
<option value="15">15</option>
<option value="30">30</option>
<option value="50">50</option>
<option selected="" value="100">100</option>
</select>
</p>
<p class="sort2">
<a href="#date1" onclick="setOrder(date); return false;"><img alt="접수일자 내림차순" id="date" name="sortImg" src="/images/sort/date/on_desc.gif"/></a>
<a href="#crp1" onclick="setOrder(crp); return false;"><img alt="회사명 내림차순" id="crp" name="sortImg" src="/images/sort/crp/sort_company_off.gif"/></a>
<a href="#rpt1" onclick="setOrder(rpt); return false;"><img alt="보고서명 내림차순" id="rpt" name="sortImg" src="/images/sort/rpt/sort_report_off.gif"/></a>
</p>
<table style="table-layout: fixed;" summary="공시서류검색에 대한 번호, 공시대상회사, 보고서명, 제출인, 접수일자, 비고 등을 알리는 표입니다.">
<caption>공시서류검색 목록</caption>
<colgroup>
<col width="6%"/>
<col width="23%"/>
<col width="*"/>
<c

#### table 코드 확인과 숫자확인

In [76]:
soup.findAll('table')

[<table style="table-layout: fixed;" summary="공시서류검색에 대한 번호, 공시대상회사, 보고서명, 제출인, 접수일자, 비고 등을 알리는 표입니다.">
 <caption>공시서류검색 목록</caption>
 <colgroup>
 <col width="6%"/>
 <col width="23%"/>
 <col width="*"/>
 <col width="15%"/>
 <col width="9%"/>
 <col width="8%"/>
 </colgroup>
 <thead>
 <tr>
 <th scope="col">번호</th>
 <th scope="col">공시대상회사</th>
 <th scope="col">보고서명</th>
 <th scope="col">제출인</th>
 <th scope="col">접수일자</th>
 <th scope="col">비고</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td class="cen_txt">
 							1
 						</td>
 <td>
 <span class="nobr1" style="max-width:150px;">
 <img src="/images/ico_kospi.gif" title="유가증권시장"/> 
 								<a href="/dsae001/selectPopup.ax?selectKey=00122348" onclick="openCorpInfo('00122348'); return false;" title="방림 기업개황 새창">
 									방림
 								</a>
 </span>
 </td>
 <td>
 <a href="/dsaf001/main.do?rcpNo=20181204800378" id="r_20181204800378" onclick="openReportViewer('20181204800378',''); return false;" title="주식분할결정 공시뷰어 새창">
 <span></span>주식분할결정 
 								

In [78]:
len(soup.findAll('table'))

1

#### 'tr' tag 첫번째, 마지막번째 요소 확인해보기

In [83]:
soup.findAll('tr')[2]

<tr>
<td class="cen_txt">
							2
						</td>
<td>
<span class="nobr1" style="max-width:150px;">
<img src="/images/ico_kosdaq.gif" title="코스닥시장"/> 
								<a href="/dsae001/selectPopup.ax?selectKey=00814786" onclick="openCorpInfo('00814786'); return false;" title="리켐 기업개황 새창">
									리켐
								</a>
</span>
</td>
<td>
<a href="/dsaf001/main.do?rcpNo=20181113900397" id="r_20181113900397" onclick="openReportViewer('20181113900397',''); return false;" title="주식분할결정 공시뷰어 새창">
<span></span>주식분할결정 
								
 								
 							</a>
</td>
<td title="리켐">
<div class="nobr" style="width:95px">리켐</div>
</td>
<td class="cen_txt">2018.11.13</td>
<td class="cen_txt end"> <img alt="본 공시사항은 한국거래소 코스닥시장본부 소관임" hspace="1" src="/images/sub/remark05.gif" title="본 공시사항은 한국거래소 코스닥시장본부 소관임"/></td>
</tr>

In [84]:
soup.findAll('tr')[1]

<tr>
<td class="cen_txt">
							1
						</td>
<td>
<span class="nobr1" style="max-width:150px;">
<img src="/images/ico_kospi.gif" title="유가증권시장"/> 
								<a href="/dsae001/selectPopup.ax?selectKey=00122348" onclick="openCorpInfo('00122348'); return false;" title="방림 기업개황 새창">
									방림
								</a>
</span>
</td>
<td>
<a href="/dsaf001/main.do?rcpNo=20181204800378" id="r_20181204800378" onclick="openReportViewer('20181204800378',''); return false;" title="주식분할결정 공시뷰어 새창">
<span></span>주식분할결정 
								
 								
 							</a>
</td>
<td title="방림">
<div class="nobr" style="width:95px">방림</div>
</td>
<td class="cen_txt">2018.12.04</td>
<td class="cen_txt end"> <img alt="본 공시사항은 한국거래소 유가증권시장본부 소관임" hspace="1" src="/images/sub/remark04.gif" title="본 공시사항은 한국거래소 유가증권시장본부 소관임"/></td>
</tr>

In [16]:
soup.findAll('tr')[-1]

<tr>
<td class="cen_txt">
							100
						</td>
<td>
<span class="nobr1" style="max-width:150px;">
<img src="/images/ico_kospi.gif" title="유가증권시장"/> 
								<a href="/dsae001/selectPopup.ax?selectKey=00167192" onclick="openCorpInfo('00167192'); return false;" title="넥센 기업개황 새창">
									넥센
								</a>
</span>
</td>
<td>
<a href="/dsaf001/main.do?rcpNo=20160223800353" id="r_20160223800353" onclick="openReportViewer('20160223800353',''); return false;" title="주식분할결정 공시뷰어 새창">
<span></span>주식분할결정 
								
 								
 							</a>
</td>
<td title="넥센">
<div class="nobr" style="width:95px">넥센</div>
</td>
<td class="cen_txt">2016.02.23</td>
<td class="cen_txt end"> <img alt="본 공시사항은 한국거래소 유가증권시장본부 소관임" hspace="1" src="/images/sub/remark04.gif" title="본 공시사항은 한국거래소 유가증권시장본부 소관임"/></td>
</tr>

#### tr tag의 숫자 확인하여 실제 공시와 비교해보기

In [85]:
len(soup.findAll('tr'))

101

## 첫번째 tr tag의 항목 추출하기

<작업 순서>

* 공시 리스트에서 공시 페이지를 여는 방법 찾아보기
  (iframe 구조 이해 필요)


* 해당 페이지에서 추출하려는 항목을 정리해야 함


* '기재정정' 공시의 구조도 같이 체크할 수 있도록 확인 필요


* 다양한 형태의 자료에 대해서도 추출 가능한 함수 제작 염두


---------------------------------------------------------

<추출 항목 정리>

 - 회사명(company)
 
 
 - 공시일자(announce_date)


 - 분할 전 액면가(far_before)


 - 분할 후 액면가(far_after)


 - 주주총회 예정일(approve_date)


 - 거래정지 첫날(close_date)


 - 신주상장일(open_date)


 - 이사회 결의일(decide_date) - 공시 시간 확인 가능

In [88]:
# tr[0]의 경우 테이블 라벨이기 때문에 1번부터 진행

work1 = soup.findAll('tr')[2].findAll('td')   

In [89]:
work1 #td_list

[<td class="cen_txt">
 							2
 						</td>, <td>
 <span class="nobr1" style="max-width:150px;">
 <img src="/images/ico_kosdaq.gif" title="코스닥시장"/> 
 								<a href="/dsae001/selectPopup.ax?selectKey=00814786" onclick="openCorpInfo('00814786'); return false;" title="리켐 기업개황 새창">
 									리켐
 								</a>
 </span>
 </td>, <td>
 <a href="/dsaf001/main.do?rcpNo=20181113900397" id="r_20181113900397" onclick="openReportViewer('20181113900397',''); return false;" title="주식분할결정 공시뷰어 새창">
 <span></span>주식분할결정 
 								
  								
  							</a>
 </td>, <td title="리켐">
 <div class="nobr" style="width:95px">리켐</div>
 </td>, <td class="cen_txt">2018.11.13</td>, <td class="cen_txt end"> <img alt="본 공시사항은 한국거래소 코스닥시장본부 소관임" hspace="1" src="/images/sub/remark05.gif" title="본 공시사항은 한국거래소 코스닥시장본부 소관임"/></td>]

#### 회사명, 공시 일자 체크 

In [90]:
work1[4].text

'2018.11.13'

In [91]:
company = work1[1].text.replace('\t','').replace('\n','').replace('\xa0','').replace('\r','')
announce_date = work1[4].text

company, announce_date


('리켐', '2018.11.13')

#### 종목코드 추출

In [92]:
code_res = requests.get('http://dart.fss.or.kr' + work1[1].a['href'])

In [93]:
code = pd.read_html(code_res.text)[0].iloc[3,1]

#### 공시 타이틀 체크('[첨부정정]'으로 시작하는 것은 지우기 위함)

In [24]:
title = work1[2].text.replace("\n", "").replace("\t","").replace(" ","").replace("\r","")
title

'주식분할결정'

#### 시장 구분 체크('일반법인 제외 목적')

In [94]:
market = work1[1].img['title']
market

'코스닥시장'

#### 하위 사이트 탐색

In [95]:
sub_url = work1[2].a['href']
sub_url_1 = 'http://dart.fss.or.kr' + sub_url
sub_url_1

'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20181113900397'

### 하위 사이트 열어보기

* sub_url_1 주소 열어서 내용 확인


* main.do 컨텐츠 확인


* viewer.do 컨텐츠 확인


* viewer.do에 원하는 컨텐츠가 있음 


* 하지만 'dcmNo' 번호에 대한 정보 필요


* 'dcmNo' 의 경우 main.do 에서 찾아보자

#### main.do 사이트 요청 및 컨텐츠 확인(html 구조가 아님)

In [27]:
sub_main_response = requests.get(sub_url_1)

In [28]:
sub_main_text = sub_main_response.text

sub_main_text

'\r\n\r\n\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n<html>\r\n<head>\r\n<title>\r\n리켐/주식분할결정/2018.11.13\r\n</title>\r\n<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8" >\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" >\r\n<style type="text/css">\r\n    .x-window-dlg .ext-mb-download {\r\n        background:transparent url(images/download.gif) no-repeat top left;\r\n        height:46px;\r\n    }\r\n</style>\r\n<link rel="stylesheet" type="text/css" href="/css/viewer.css" charset="utf-8">\r\n<script type="text/javascript" src="/js/prototype.js"></script>\r\n<script type="text/javascript" src="/js/jquery/jquery-all.js"></script>\r\n\r\n<!-- 2014.03.11 ext 3.4 -->\r\n<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-viewer/resources/css/ext-all-ie8.css" ><![endif]-->\r\n<!--[if (gte IE 9)|!(IE)]><!--><link rel="stylesheet" type="text/css" href="/js/ext-viewer/resources/c

#### dcmNo 위치를 확인해보자.(ctrl + F 명령어로 검색)

#### dcmNo 위치와 관련하여 반복되는 구문을 찾아보자(201804.......)

#### 2018042.....넘버를 먼저 구한 후 이를 중심으로 split하여 두번째 항목을 살펴보자.

Hint) sub_url의 마지막 부분에 있음

In [96]:
check_num = sub_url[-14:]
check_num

'20181113900397'

In [97]:
sub_main_text.split(check_num)[1]

'\', \'6378012\', null, null, null, \'HTML\')"\r\n\t});\r\n\r\n\twest = new Tree.TreePanel({\r\n\t\tid:"west-panel",\r\n\t\ttitle:"문서목차",\r\n\t\tregion:"west",\r\n\t\tsplit:true,\r\n\t\tautoScroll:true,\r\n\t\twidth: 200,\r\n\t\tminSize: 175,\r\n\t\tmaxSize: 400,\r\n\t\tcollapsible: true,\r\n\t\tcollapsed: false,\r\n\t\tmargins:"5 0 5 5",\r\n\t\tanimate:false,\r\n\t\tuseArrows:false,\r\n\t\tloader: new Tree.TreeLoader(),\r\n\t\tenableDD:false,\r\n\t\tcontainerScroll: true,\r\n\t\tchecked: false,\r\n\t\trootVisible: false,\r\n\t\ttitleCollapse: false\r\n\t});\r\n\r\n\tvar cnt = 0;\r\n\r\n\t\r\n\t\r\n\r\n\twest.setRootNode(treeRoot);\r\n\r\n\tvar viewport = new Ext.Viewport({\r\n\t     layout:\'border\',\r\n\t     items:[\r\n\t     \tnorth,\r\n\t     \t\r\n\t     \tnew Ext.Panel({\r\n\t             region:\'center\',\r\n\t             id:\'center-panel\',\r\n\t             margins:\'5 5 5 5\',\r\n\t             autoScroll:false,\r\n\t             items: [{\r\n\t                 contentEl

#### 원하는 번호가 있는 위치를 찾아보자.(노가다로)

In [98]:
sub_main_text.split(check_num)[1][4:12]

"6378012'"

#### 원하는 번호가 있는 위치를 찾아보자.(정규표현식으로)

* 7자리 숫자를 찾아주는 정규표현식은?


* 'python regrex exact n digit number' 로 구글에서 검색


* 정규표현식 문법은 네이버에서 '정규표현식'으로 검색해볼 것

In [99]:
re.findall(r"\b\d{7}\b",sub_main_text.split(check_num)[1])[0]

'6378012'

In [100]:
dcmNo = re.findall(r"\b\d{7}\b",sub_main_text.split(check_num)[1])[0]
dcmNo

'6378012'

### dcmNo를 활용하여 viewer.do 문서를 열어보자.

In [101]:
contents_url = "http://dart.fss.or.kr/report/viewer.do"

#### requests.get의 parameter check

In [102]:
param_data = {
    'rcpNo': check_num,    # check_num 사용(앞에서 추출한 고유번호)
    'dcmNo': dcmNo,        # dcmNo 사용(앞에서 추출한 고유번호)
    'eleId': '0',
    'offset': '0',
    'length': '0',
    'dtd': 'HTML'
}

#### param을 활용하여 requests 하기

In [103]:
contents_request = requests.get(contents_url, params = param_data)

In [104]:
contents_request.text

'\r\n<html>\r\n <head><link href="/css/koscom_web2.css" rel="stylesheet" type="text/css"/>\r\n  <META content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>\r\n  <meta content="text/html; charset=euc-kr" http-equiv="Content-Type"/>\r\n  <STYLE>\r\n.xforms * { font-family: 돋움체;}\r\n.xforms_title * { font-size: 13pt; padding : 0 0 10 0;}\r\n\r\n.xforms table {\r\n    font-size: 10px;\r\n    padding:0px;\r\n    border-collapse:collapse;\r\n    color: #3D3D3D;\r\n}\r\n.xforms td {\r\n    padding-left:0px;\r\n    padding-top:0px;\r\n    border-collapse:collapse;\r\n    line-height:22px;\r\n    color: #3D3D3D;\r\n    /*text-align:right;*/\r\n    vertical-align:middle;\r\n    border: 1px solid #7f7f7f;\r\n}\r\n\r\n.xforms span {\r\n    /*display: inline-block;*/\r\n    padding-left:3px;\r\n    padding-right: 1px;\r\n    line-height:22px;\r\n    padding-top:0px;\r\n    padding-bottom:0px;\r\n    text-align:left;\r\n    vertical-align:middle;\r\n    border: 0px solid #DBDBDB;\r\n}\r\n\r\n.xforms.i

#### 화면 구조 분석

In [105]:
contents_df = pd.read_html(contents_request.text)


In [106]:
contents_df[-1]

,0,1,2,3
0,1. 주식분할 내용,구분,분할전,분할후
1,1주당 가액 (원),500,100,NaN
2,발행주식총수,보통주식(주),4553965,22769825
3,종류주식(주),-,-,NaN
4,2. 주식분할 일정,주주총회예정일,2018-12-27,NaN
5,구주권제출기간,시작일,2018-12-28,NaN
6,종료일,2019-01-29,NaN,NaN
7,매매거래정지예정기간,2019-01-28 ~ 2019-02-14,NaN,NaN
8,명의개서정지기간,2019-01-30 ~ 2019-02-13,NaN,NaN
9,신주권교부예정일,2019-02-14,NaN,NaN


In [107]:
len(contents_df)

1

----------


### <추출 항목 정리>

 - 회사명(company)
 
 
 - 공시일자(announce_date)


 - 분할 전 액면가(far_before)


 - 분할 후 액면가(far_after)


 - 주주총회 예정일(approve_date)


 - 거래정지 첫날(close_date)


 - 신주상장일(open_date)


 - 이사회 결의일(decide_date) - 공시 시간 확인 가능

------

### (매우 중요) 주요 명심사항!

* 이번 document 뿐만 아니라 다른 공시에 대해서도 적용이 되어야 함!


* 경우의 수를 잘 생각해 보자.


* 지금 최대한 만들어도 나중에 바꿔야할 가능성 99.999% 이니 일단 한번 해보고 수정하자.

---



1) 정정 공시일 경우 관련 table 이 추가됨


2) 매매거래정지기간의 경우 날짜 양식이 다를 경우


3) 정정 신고 내용에 주요 항목이 표시된 경우


4) 항목이 다른 경우

#### (매우 중요) table 항목을 찾아 가장 마지막 item을 선택하여 작업하도록 함

* 정정 공시의 경우 정정 이전 또는 정정 관련 사항이 가장 위에 위치함


* 마지막 table을 선택하여 작업할 경우 정정 이후 공시만 추출할 수 있음

In [41]:
contents_df[-1]

,0,1,2,3
0,1. 주식분할 내용,구분,분할전,분할후
1,1주당 가액 (원),500,100,NaN
2,발행주식총수,보통주식(주),4553965,22769825
3,종류주식(주),-,-,NaN
4,2. 주식분할 일정,주주총회예정일,2018-12-27,NaN
5,구주권제출기간,시작일,2018-12-28,NaN
6,종료일,2019-01-29,NaN,NaN
7,매매거래정지예정기간,2019-01-28 ~ 2019-02-14,NaN,NaN
8,명의개서정지기간,2019-01-30 ~ 2019-02-13,NaN,NaN
9,신주권교부예정일,2019-02-14,NaN,NaN


In [108]:
far_before = int(contents_df[-1][contents_df[-1][0].str.contains('1주당')][1].values[0])
far_before

500

In [109]:
far_after = int(contents_df[-1][contents_df[-1][0].str.contains('1주당')][2].values[0])
far_after

100

In [110]:
approve_date = contents_df[-1][contents_df[-1][0].str.contains('일정')][2].values[0]
approve_date

'2018-12-27'

In [111]:
announce_date = contents_df[-1][contents_df[-1][0].str.contains('이사회')][1].values[0]
announce_date

'2018-11-13'

#### 거래정지 첫날(close_date)

* 이 경우 매매거래 정지기간은 표기 양식이 공시에 따라 너무 다름


* 구주권 제출 기간 -종료일 을 선택할 경우 마지막 거래일을 추출할 수 있음


In [112]:
close_date = contents_df[-1][contents_df[-1][0].str.contains('종료일')][1].values[0]
close_date

'2019-01-29'

In [114]:
open_date = contents_df[-1][contents_df[-1][0].str.contains('상장')][1].values[0]
open_date

'2019-02-15'

### 해당 공시 정보 출력해보기

In [115]:
print(company, code, announce_date,far_before, far_after, approve_date,close_date,open_date)

리켐 131100 2018-11-13 500 100 2018-12-27 2019-01-29 2019-02-15


-----

# 2016년 이후 전체 액면분할 공시 출력해보기

### 전체 진행 순서

1) 공시 리스트 불러오기


2) 리스트에서 일부 항목 추출 


3) 각 공시의 서브페이지 중 컨텐츠 페이지 불러오기


4) 컨텐츠 페이지에서 주요 항목 추출


5) 추출한 내용을 dictionary 형식으로 저장



----

### 1) 공시 리스트 불러오기

#### 금일 날짜 설정

In [116]:
dt = datetime.utcnow() + timedelta(hours = 9)    # utc 시간에서 9시간 더함
today = dt.strftime('%Y%m%d')                  # str 형태로 변환
today

'20190105'

#### 리스트 url 찾기

In [117]:
url = 'http://dart.fss.or.kr/dsab002/search.ax'

#### data parameter 구하기

In [118]:
data = {
    'currentPage': '1',            # 페이지 1로 충분함
    'maxResults': '100',           # 15에서 100으로 변경
    'maxLinks': '10',
    'sort': 'date',
    'series': 'desc',
    'textCrpCik': '',
    'textCrpNm': '',
    'textPresenterNm': '',
    'startDate': '20160101',
    'endDate': today,         # 오늘 날짜로 변경
    'finalReport': 'recent',
    'typesOfBusiness': 'all',
    'corporationType': 'all',
    'closingAccountsMonth': 'all',
    'reportName': '주식분할'
    }

#### 원하는 페이지 요청

In [119]:
response = requests.post(url, data = data)

#### 페이지 응답여부 확인해 보기

In [120]:
pd.read_html(response.text)[0]

,번호,공시대상회사,보고서명,제출인,접수일자,비고
0,1,방림,주식분할결정,방림,2018.12.04,NaN
1,2,리켐,주식분할결정,리켐,2018.11.13,NaN
2,3,중앙에너비스,[기재정정]주식분할결정,중앙에너비스,2018.10.30,NaN
3,4,포스링크,[기재정정]주식분할결정,포스링크,2018.10.19,NaN
4,5,인콘,[기재정정]주식분할결정,인콘,2018.09.11,NaN
5,6,메가스터디교육,[기재정정]주식분할결정,메가스터디교육,2018.08.22,NaN
6,7,모트렉스,[기재정정]주식분할결정,모트렉스,2018.08.09,NaN
7,8,바이오빌,[기재정정]주식분할결정,바이오빌,2018.08.07,NaN
8,9,대창솔루션,[첨부정정]주식분할결정,대창솔루션,2018.08.01,NaN
9,10,에이스침대,주식분할결정,에이스침대,2018.08.01,NaN


#### 페이지 파싱(BeautifulSoup 활용)

In [121]:
soup = bs(response.text, 'html.parser')

#### tr_list 설정

In [122]:
tr_list = soup.findAll('tr')

#### text 에서 불필요한 문자 remove하는 함수 정의

In [123]:
def remove_text(item) :
    return item.replace('\n', "").replace('\t', "").replace(',' ,'').replace('\xa0', '').replace('\r', '')

공시 item에서 띄어쓰기를 없애주는 함수 정의

* 공시 item이 띄어쓰기 통일이 안되어 있음

예시) "1주당 가액 (원)", "1주당 가액(원)"

In [124]:
def del_space(item) :
    return item.replace(" ", "")

#### 각 tr별로 주요 컨텐츠 추출하는 for 문 작성

In [125]:
result_list = []     # 결과를 담을 빈 리스트 생성

for tr in progressbar(tr_list[1:]) :         # tr리스트의 1번째는 컬럼명임. 그래서 두번째부터 진행
    
    td_list = tr.findAll('td')  # 각 tr item 내에서 td 리스트를 생성
    except_list = ['메디플란트', '바이오빌', '포스링크']   # 분할 철회 등으로 빼야하는 리스트 생성
    
    
    
    company = remove_text(td_list[1].text)
    code_res = requests.get('http://dart.fss.or.kr' + td_list[1].a['href'])
    code = "A" + pd.read_html(code_res.text)[0].iloc[3,1]
    title = remove_text(td_list[2].text)
    market = td_list[1].img['title']
    
    
    if ('첨부정정' in title) or ('기타법인' in market) or (company in except_list) :
        pass 
    
    else : 
        sub_url = 'http://dart.fss.or.kr' + td_list[2].a['href']


        # 하위 페이지 요청하기
        sub_response = requests.get(sub_url)
        sub_text = sub_response.text

        # url 마지막에 있는 문서번호 구하기
        check_num = str(sub_url[-14:])


        # dcmNo 구하기 
        dcmNo = re.findall(r"\b\d{7}\b",sub_text.split(check_num)[1])[0]

        # contents가 포함된 html 문서 불러오기
        contents_url = "http://dart.fss.or.kr/report/viewer.do"
        param_data = {
        'rcpNo': check_num,    # check_num 사용(앞에서 추출한 고유번호)
        'dcmNo': dcmNo,        # dcmNo 사용(앞에서 추출한 고유번호)
        'eleId': '0',
        'offset': '0',
        'length': '0',
        'dtd': 'HTML'
        }

        contents_request = requests.get(contents_url, params = param_data)
        contents_df = pd.read_html(contents_request.text)
        
        # 각 항목 추출하기

        far_before = int(contents_df[-1][contents_df[-1][0].str.contains('1주당')][1].values[0])
        far_after = int(contents_df[-1][contents_df[-1][0].str.contains('1주당')][2].values[0])
        announce_date = contents_df[-1][contents_df[-1][0].str.contains('이사회')][1].values[0]
        approve_date = contents_df[-1][contents_df[-1][0].str.contains('일정')][2].values[0]
        close_date = contents_df[-1][contents_df[-1][0].str.contains('종료일')][1].values[0]
        open_date = contents_df[-1][contents_df[-1][0].str.contains('상장')][1].values[0]
        
        result_list.append({
            '코드' : code,
            '회사명' : company, 
            '발표일' : announce_date,
            '액면가(전)' : far_before,
            '액면가(후)' : far_after, 
            '주주총회일' : approve_date,
            '거래정지일' : close_date,
            '거래재개일' : open_date
        })

        time.sleep(random.randint(5,10)/10)

100% (100 of 100) |######################| Elapsed Time: 0:02:29 Time:  0:02:29


In [60]:
result_list

[{'코드': 'A003610',
  '회사명': '방림',
  '발표일': '2018-12-04',
  '액면가(전)': 5000,
  '액면가(후)': 500,
  '주주총회일': '2018-12-21',
  '거래정지일': '2019-01-25',
  '거래재개일': '2019-01-29'},
 {'코드': 'A131100',
  '회사명': '리켐',
  '발표일': '2018-11-13',
  '액면가(전)': 500,
  '액면가(후)': 100,
  '주주총회일': '2018-12-27',
  '거래정지일': '2019-01-29',
  '거래재개일': '2019-02-15'},
 {'코드': 'A000440',
  '회사명': '중앙에너비스',
  '발표일': '2018-09-07',
  '액면가(전)': 2500,
  '액면가(후)': 500,
  '주주총회일': '2018-10-30',
  '거래정지일': '2018-12-04',
  '거래재개일': '2018-12-06'},
 {'코드': 'A083640',
  '회사명': '인콘',
  '발표일': '2018-09-11',
  '액면가(전)': 500,
  '액면가(후)': 100,
  '주주총회일': '2018-07-31',
  '거래정지일': '2018-09-03',
  '거래재개일': '2018-10-10'},
 {'코드': 'A215200',
  '회사명': '메가스터디교육',
  '발표일': '2018-05-28',
  '액면가(전)': 500,
  '액면가(후)': 100,
  '주주총회일': '2018-07-13',
  '거래정지일': '2018-08-17',
  '거래재개일': '2018-08-29'},
 {'코드': 'A118990',
  '회사명': '모트렉스',
  '발표일': '2018-08-09',
  '액면가(전)': 500,
  '액면가(후)': 100,
  '주주총회일': '2018-07-18',
  '거래정지일': '2018-08-20',
  '거래재개일': 

In [61]:
work_df = pd.DataFrame(result_list)
work_df

,거래재개일,거래정지일,발표일,액면가(전),액면가(후),주주총회일,코드,회사명
0,2019-01-29,2019-01-25,2018-12-04,5000,500,2018-12-21,A003610,방림
1,2019-02-15,2019-01-29,2018-11-13,500,100,2018-12-27,A131100,리켐
2,2018-12-06,2018-12-04,2018-09-07,2500,500,2018-10-30,A000440,중앙에너비스
3,2018-10-10,2018-09-03,2018-09-11,500,100,2018-07-31,A083640,인콘
4,2018-08-29,2018-08-17,2018-05-28,500,100,2018-07-13,A215200,메가스터디교육
5,2018-08-22,2018-08-20,2018-08-09,500,100,2018-07-18,A118990,모트렉스
6,2018-10-22,2018-10-18,2018-08-01,5000,1000,2018-09-14,A003800,에이스침대
7,2018-10-12,2018-10-10,2018-07-25,500,100,2018-09-07,A035420,NAVER
8,2018-07-16,2018-06-28,2018-03-12,500,100,2018-03-27,A033430,디에스티
9,2018-09-04,2018-08-21,2018-07-02,500,100,2018-07-16,A102210,해덕파워웨이


In [ ]:
work_df.columns

In [ ]:
col = ['코드','회사명','액면가(전)','액면가(후)','발표일','주주총회일','거래정지일','거래재개일']

In [ ]:
work_df = work_df[col]

In [ ]:
work_df = work_df.set_index('코드')

In [ ]:
work_df

### Nan 값 추출하기

In [ ]:
work_df.isnull().sum()


### 특정 값 포함했는지 확인하기

In [ ]:
work_df.isin(['-']).sum()

### 특정값 포함한 것 확인하기

In [ ]:
work_df[work_df['주주총회일'].isin(['-'])]

### 수동으로 입력하기

In [ ]:
work_df.loc[work_df['회사명']=='대양제지', '주주총회일'] = '2017-03-25'

work_df.loc[work_df['회사명']=='코리아에스이', '주주총회일'] = '2016-03-24'

In [ ]:
work_df[work_df['회사명'].isin(['대양제지', '코리아에스이'])]

### 엑셀로 보내기

In [ ]:
#work_df.to_excel('액면분할.xlsx', encoding = 'euckr')

### 엑셀에서 읽어오기

In [ ]:
work = pd.read_excel("액면분할.xlsx", encoding = 'euckr', index_col = '코드')

In [ ]:
work

In [ ]:
work[['주주총회일', '거래정지일', '거래재개일']] = work[['주주총회일', '거래정지일', '거래재개일']].apply(pd.to_datetime)

In [ ]:
work.info()